In [18]:
import requests
import pandas as pd
from time import sleep
import numpy as np
import os
import re
import calendar

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException


import warnings
warnings.filterwarnings('ignore')

#### Web Scraping

In [2]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized')
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito')

In [26]:
años = ["2016", "2017", "2018", "2019", "2020", "2021", "2022"]
meses = ["january", "february", "march", "april", "may", "june",
         "july", "august", "september", "octuber", "november", "december"]

In [32]:
driver = webdriver.Chrome(ChromeDriverManager().install())
for año in años:
    for mes in meses:
        if mes == "january" and año == "2016":
            driver.implicitly_wait(20)
            driver.get(f"https://sullygnome.com/games/{año}{mes}/watched")
            driver.find_element("css selector", "#tblControl_length > label > select").click()
            sleep(2)
            driver.find_element("css selector", "#tblControl_length > label > select > option:nth-child(4)").click()
            sleep(2)
            resultado = driver.find_element("css selector","#tblControl_wrapper > div.top.TableTopMenu > div.dt-buttons > button.dt-button.buttons-csv.buttons-html5.TableExportLinkButton").click()
            sleep(3)
            os.rename(rf"C:\Users\jorge\Downloads\Most watched games on Twitch - SullyGnome.csv",
            rf"C:\Users\jorge\Downloads\Most watched games on Twitch {año} {mes} - SullyGnome.csv")
        else:
            driver.implicitly_wait(20)
            driver.get(f"https://sullygnome.com/games/{año}{mes}/watched")
            sleep(3)
            resultado = driver.find_element("css selector","#tblControl_wrapper > div.top.TableTopMenu > div.dt-buttons > button.dt-button.buttons-csv.buttons-html5.TableExportLinkButton").click()
            sleep(3)
            os.rename(rf"C:\Users\jorge\Downloads\Most watched games on Twitch - SullyGnome.csv",
            rf"C:\Users\jorge\Downloads\Most watched games on Twitch {año} {mes} - SullyGnome.csv")

In [ ]:
for año in años:
    for mes in meses:
        df = pd.dataframe()

In [70]:
contenido = os.listdir(r"C:\Users\jorge\OneDrive\Escritorio\IronHack\Work\Proyecto_1\CSV Twitch")
archivos = []
for nombre in contenido:
    archivos.append(re.findall("\d{4}\s\w+\s", nombre)[0].rstrip().split(" "))
archivos[0:3]

[['2016', 'april'], ['2016', 'august'], ['2016', 'december']]

In [78]:
lista_archivos = []
for archivo in archivos:
    df = pd.read_csv(f"CSV Twitch\Most watched games on Twitch {archivo[0]} {archivo[1]} - SullyGnome.csv")
    df.drop(["Unnamed: 0", "Unnamed: 1", "Unnamed: 11"], axis=1, inplace=True)
    df["Año"] = archivo[0]
    df["Mes"] = archivo[1]
    lista_archivos.append(df)

In [92]:
df_twitch = pd.concat(lista_archivos, axis=0, ignore_index=True)

In [118]:
df_twitch.to_csv("twitch_data")

#### Data Wrangling & Data Cleaning

In [118]:
df = pd.read_csv("twitch_data")
df.drop(["Unnamed: 0"], axis=1,inplace=True)

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8400 entries, 0 to 8399
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Game                  8400 non-null   object 
 1   Watch time (mins)     8400 non-null   int64  
 2   Stream time (mins)    8400 non-null   int64  
 3   Peak viewers          8400 non-null   int64  
 4   Peak channels         8400 non-null   int64  
 5   Streamers             8400 non-null   int64  
 6   Average viewers       8400 non-null   int64  
 7   Average channels      8400 non-null   int64  
 8   Average viewer ratio  8400 non-null   float64
 9   Año                   8400 non-null   int64  
 10  Mes                   8400 non-null   object 
dtypes: float64(1), int64(8), object(2)
memory usage: 722.0+ KB


In [120]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Watch time (mins),8400.0,1.131352e+09,5.381248e+09,14506080.00,75070110.00,1.795218e+08,5.582556e+08,1.883467e+11
Stream time (mins),8400.0,3.550903e+07,1.636521e+08,7380.00,2185005.00,5.821170e+06,1.827904e+07,4.965171e+09
Peak viewers,8400.0,1.099820e+05,2.275898e+05,696.00,18781.75,4.078050e+04,1.000890e+05,3.366021e+06
Peak channels,8400.0,1.298094e+03,4.367347e+03,3.00,132.00,3.100000e+02,9.535000e+02,1.298600e+05
Streamers,8400.0,5.095480e+04,1.565993e+05,0.00,4215.00,1.089650e+04,3.673725e+04,4.058092e+06
Average viewers,8400.0,1.369172e+04,3.395789e+04,336.00,1574.00,3.599000e+03,8.425250e+03,4.792090e+05
Average channels,8400.0,4.282768e+02,1.037898e+03,0.00,46.00,1.140000e+02,2.982500e+02,1.378900e+04
Average viewer ratio,8400.0,5.775625e+01,1.407767e+02,1.95,18.42,3.235500e+01,6.109000e+01,4.586720e+03
Año,8400.0,2.019000e+03,2.000119e+00,2016.00,2017.00,2.019000e+03,2.021000e+03,2.022000e+03


In [121]:
df.describe(include="object").T

,count,unique,top,freq
Game,8400,936,League of Legends,84
Mes,8400,12,april,700


In [122]:
df.head(1)

,Game,Watch time (mins),Stream time (mins),Peak viewers,Peak channels,Streamers,Average viewers,Average channels,Average viewer ratio,Año,Mes
0,League of Legends,5303342940,73035000,553165,2945,113251,122933,1692,72.61,2016,april


In [123]:
df.Mes = df.Mes.str.replace("octuber", "october")

In [124]:
mes_n = ["01","02","03","04","05","06","07","08","09","10","11","12"]
lista_meses = list(calendar.month_name)[1:]
mes_c = []
for mes in lista_meses:
    mes_c.append(mes.lower())
mes_c

['january',
 'february',
 'march',
 'april',
 'may',
 'june',
 'july',
 'august',
 'september',
 'october',
 'november',
 'december']

In [125]:
def meses(m):
    for n, c in zip(mes_n, mes_c):
        if m == c:
            return n

In [126]:
df["mes"] = df["Mes"].apply(meses)

In [127]:
df.sample(2)

,Game,Watch time (mins),Stream time (mins),Peak viewers,Peak channels,Streamers,Average viewers,Average channels,Average viewer ratio,Año,Mes,mes
497,Secret Hitler,21206100,29520,41675,12,112,475,0,718.36,2016,january,01
2608,Atlas,1160434620,9056640,318887,2104,14786,26030,203,128.13,2018,december,12


In [128]:
df["fecha"] = df["Año"].astype("str") + "-" + df["mes"].astype("str")


In [129]:
df.drop(["Año", "mes", "Mes"], axis=1, inplace=True)

In [130]:
df.head(1)

,Game,Watch time (mins),Stream time (mins),Peak viewers,Peak channels,Streamers,Average viewers,Average channels,Average viewer ratio,fecha
0,League of Legends,5303342940,73035000,553165,2945,113251,122933,1692,72.61,2016-04


In [132]:
df.columns = list(map(lambda x: x.replace(" ","_").lower(), df.columns))

In [133]:
df.head(1)

,game,watch_time_(mins),stream_time_(mins),peak_viewers,peak_channels,streamers,average_viewers,average_channels,average_viewer_ratio,fecha
0,League of Legends,5303342940,73035000,553165,2945,113251,122933,1692,72.61,2016-04


In [134]:
df["watch_time_(hours)"] = df["watch_time_(mins)"] / 60
df["stream_time_(hours)"] = df["stream_time_(mins)"] / 60

In [135]:
df.drop(["watch_time_(mins)", "stream_time_(mins)"], axis=1, inplace=True)

In [136]:
df = df[["game", "watch_time_(hours)", "stream_time_(hours)", "peak_viewers", "peak_channels",
              "streamers", "average_viewers", "average_channels", "average_viewer_ratio","fecha"]]

In [137]:
df.head(1)

,game,watch_time_(hours),stream_time_(hours),peak_viewers,peak_channels,streamers,average_viewers,average_channels,average_viewer_ratio,fecha
0,League of Legends,88389049.0,1217250.0,553165,2945,113251,122933,1692,72.61,2016-04
